### add "fraudulent" = 0 for original real data

In [1]:
import pandas as pd


df = pd.read_csv("Jadarat_data.csv", encoding="utf-8")


if "fraudulent" not in df.columns:
    df.insert(0, "fraudulent", 0)
else:
    df["fraudulent"] = 0

df.head()

,fraudulent,job_title,job_date,job_desc,job_tasks,comp_name,comp_no,comp_type,comp_size,eco_activity,qualif,region,city,benefits,contract,positions,job_post_id,exper,gender
0,0,محاسب,27/05/1444,['إعداد وتنظيم مستندات الصرف ومتابعة تحصيل الإ...,[' تدقيق المطالبات المالية والتأكد من اكتمال...,شركة مقر العالم للسفريات,1-317262,خاص,متوسطة فئة أ,أنشطة وكالات السياحة والسفر,"['Language data', 'اللغة الانجليزية', 'محترف']",الرياض,AR RIYADH...,"['Salary', '5000.0']",دوام كامل,0 / 1,20202026350419,0 Years,both
1,0,بائع,27/05/1444,['بيع مجموعة من السلع والخدمات للعملاء، وتوفير...,"[' بيع مجموعة من السلع والخدمات للعملاء.', '...",شركة عالم الكهرباء للمقاولات,4-1324428,خاص,متوسطة فئة ب,تركيب انظمة التبريد وتكييف الهواء وصيانتها واص...,NaN,المنطقة الشرقية,AD DAMMAM...,"['Salary', '5000.0']",دوام كامل,0 / 3,20202026350389,0 Years,both
2,0,أخصائي عمليات موارد بشرية,27/05/1444,['تنفيذ الإجراءات والأنظمة والنماذج الخاصة بمر...,[' تنفيذ الإجراءات والأنظمة والنماذج الخاصة ...,شركه دار السلام,1-155294,خاص,متوسطة فئة أ,ترميمات المباني السكنية والغير سكنية,"['Language data', 'الانجليزيه', 'محترف']",الرياض,ATH THUMA...,"['Salary', '4000.0']",دوام كامل,0 / 2,20202026350347,2 Years,both
3,0,ميكانيكي سيارات,27/05/1444,['تشخيص أعطال السيارات وإصلاحها وتنفيذ برامج ا...,[' فحص أداء المعدّات الكهربائية والميكانيكية...,مؤسسة لمكو لغيار الزيوت,8-1925495,خاص,صغيرة فئة ب,NaN,"['Skill data', 'صيانة السيارات وتقييم الاعطال'...",المنطقة الشرقية,AL HUFUF...,"['Salary', '5000.0']",دوام كامل,0 / 10,20202026350219,0 Years,M
4,0,محاسب,27/05/1444,['إعداد وتنظيم مستندات الصرف ومتابعة تحصيل الإ...,[' تدقيق المطالبات المالية والتأكد من اكتمال...,مؤسسة فكرة اليمامة للمقاولات,1-2356639,خاص,كبيرة,الإنشاءات العامة للمباني السكنية,"['Skill data', 'تحمل ضغط العمل', 'محترف', 'Lan...",الرياض,AR RIYADH...,"['Salary', '5000.0']",دوام كامل,0 / 1,20202026350043,0 Years,both


### Merge Fake with Real data

In [2]:
import pandas as pd
import glob
import os


main_csv = "Jadarat_data.csv"
main_df = pd.read_csv(main_csv, encoding="utf-8-sig")


if "fraudulent" not in main_df.columns:
    main_df["fraudulent"] = 0


json_files = glob.glob("FakeData/*.json")

fake_dfs = []
for jf in json_files:
    try:
        fake_df = pd.read_json(jf, encoding="utf-8")
        if "fraudulent" not in fake_df.columns:
            fake_df["fraudulent"] = 1
        fake_dfs.append(fake_df)
        print(f"Loaded JSON: {jf} -> {fake_df.shape}")
    except Exception as e:
        print(f"Skipped {jf}: {e}")

if not fake_dfs:
    raise RuntimeError("لم يتم العثور على ملفات JSON صالحة في المجلد FakeData.")


preferred_cols = [
    "fraudulent","job_title","job_date","job_desc","job_tasks",
    "comp_name","comp_no","comp_type","comp_size","eco_activity",
    "qualif","region","city","benefits","contract","positions",
    "job_post_id","exper"
]

all_fake = pd.concat(fake_dfs, ignore_index=True)
all_cols_union = list(dict.fromkeys(preferred_cols + list(main_df.columns) + list(all_fake.columns)))

main_aligned = main_df.reindex(columns=all_cols_union)
fake_aligned = all_fake.reindex(columns=all_cols_union)


combined_df = pd.concat([main_aligned, fake_aligned], ignore_index=True)
rest_cols = [c for c in combined_df.columns if c not in preferred_cols]
combined_df = combined_df[preferred_cols + rest_cols]
combined_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)


out_csv = "mergedFakeWithRealData.csv"
combined_df.to_csv(out_csv, index=False, encoding="utf-8-sig")


fraud_count = (combined_df["fraudulent"] == 1).sum() if "fraudulent" in combined_df.columns else 0
real_count  = (combined_df["fraudulent"] == 0).sum() if "fraudulent" in combined_df.columns else 0

print("Merge done.")
print(f"Final shape: {combined_df.shape}")
print(f"Legitimate rows (fraudulent=0): {real_count}")
print(f"Fake rows (fraudulent=1): {fraud_count}")
print(f"Saved to: {os.path.abspath(out_csv)}")


Loaded JSON: FakeData\benefits_contract_mismatch.json -> (50, 18)
Loaded JSON: FakeData\compsize_benefits_mismatch.json -> (50, 18)
Loaded JSON: FakeData\contract_jobtasks_mismatch.json -> (50, 18)
Loaded JSON: FakeData\ecoactivity_jobtasks_mismatch.json -> (50, 18)
Loaded JSON: FakeData\jobdate_jobdesc_mismatch.json -> (50, 18)
Loaded JSON: FakeData\jobdesc_gender_mismatch.json -> (50, 16)
Loaded JSON: FakeData\jobtasks_exper_mismatch.json -> (1, 15)
Loaded JSON: FakeData\jobtitle_comptype_mismatch.json -> (50, 18)
Loaded JSON: FakeData\jobtitle_exper_mismatch.json -> (50, 18)
Loaded JSON: FakeData\jobtitle_jobdesc_mismatch.json -> (50, 18)
Loaded JSON: FakeData\jobtitle_qualif_mismatch.json -> (50, 18)
Loaded JSON: FakeData\qualif_jobtitle_mismatch.json -> (2, 15)
Loaded JSON: FakeData\region_city_mismatch.json -> (50, 18)
Merge done.
Final shape: (2023, 19)
Legitimate rows (fraudulent=0): 1470
Fake rows (fraudulent=1): 553
Saved to: c:\Users\Lolo-\Downloads\machine-learning-project-

### To double check if Fake is exist

In [3]:
# extract only Fake data
fraud_df = combined_df[combined_df["fraudulent"] == 1]


import IPython.display as disp
disp.display(fraud_df)


print("Fake rows number", len(fraud_df))


,fraudulent,job_title,job_date,job_desc,job_tasks,comp_name,comp_no,comp_type,comp_size,eco_activity,qualif,region,city,benefits,contract,positions,job_post_id,exper,gender
10,1,خدمة عملاء,27/05/1444,['فرصة عمل مميزة براتب مغري.'],"[' وضع استراتيجيات طويلة الأمد', ' تطوير خ...",جهة حكومية سرية,5-292433,حكومي سري,كبيرة,NaN,NaN,الرياض,JEDDAH...,"['Salary','30000']",عقد مؤقت,0 / 35,FRAUD2025CONTASK1045,1 Years,NaN
19,1,كاتب محتوى,30/05/1444,['وظيفة عاجلة - تواصل عبر البريد الشخصي.'],"[' ترتيب المنتجات', ' خدمة الزبائن', ' م...",جهة حكومية سرية,1-725396,حكومي سري,صغيرة فئة أ,Education,NaN,المنطقة الشرقية,AD DAMMAM...,"['Salary','20000']",دوام كامل,0 / 3,FRAUD2025ECO1015,1 Years,NaN
20,1,مسوق إلكتروني,27/05/1444,['راتب عالي جدا بدون أي متطلبات إضافية.'],"[' فحص المرضى', ' كتابة تقارير طبية', ' ...",مؤسسة النجاح الفوري,9-878394,حكومي سري,كبيرة,Information Technology,NaN,المنطقة الشرقية,AD DAMMAM...,"['Salary','30000']",دوام كامل,0 / 12,FRAUD2025ECO1006,0 Years,NaN
26,1,سكرتير,01/01/1438,['فرصة محدودة - التحق فوراً قبل إغلاق التقديم.'],"[' الرد على رسائل واتساب', ' حفظ ملفات بسي...",جهة حكومية سرية,3-698181,خاص,صغيرة فئة أ,NaN,NaN,مكة المكرمة,AR RIYADH...,"['Salary','15000']",دوام كامل,0 / 18,FRAUD2025DATEDESC1040,0 Years,NaN
28,1,محاسب,27/05/1444,['فرصة عمل مميزة براتب مغري.'],"[' الرد على رسائل واتساب', ' حفظ ملفات بسي...",جهة حكومية سرية,1-614106,خاص,متوسطة فئة أ,NaN,NaN,الرياض,AL MADINAH...,"['Salary','90000']",عقد مؤقت,0 / 13,FRAUD2025SIZEBEN1034,1 Years,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2002,1,خدمة عملاء,20/02/1439,['مطلوب فورا! آخر موعد للتقديم اليوم.'],"[' الرد على رسائل واتساب', ' حفظ ملفات بسي...",مؤسسة النجاح الفوري,2-268270,خاص,متوسطة فئة أ,NaN,NaN,المنطقة الشرقية,JEDDAH...,"['Salary','25000']",دوام جزئي,0 / 21,FRAUD2025DATEDESC1015,3 Years,NaN
2004,1,مهندس مدني,20/02/1439,['مقابلة فورية - الحضور خلال 24 ساعة.'],"[' مهام مكتبية بسيطة', ' الرد على العملاء'...",شركة الثروة السريعة,3-623476,خاص,متوسطة فئة أ,NaN,NaN,المدينة المنورة,AR RIYADH...,"['Salary','25000']",دوام جزئي,0 / 8,FRAUD2025DATEDESC1017,5 Years,NaN
2009,1,مسوق إلكتروني,29/05/1444,['وظيفة عاجلة - تواصل عبر البريد الشخصي.'],"[' إعداد تقارير', ' متابعة البريد', ' ال...",شركة الثروة السريعة,1-752092,خاص,صغيرة فئة أ,NaN,NaN,مكة المكرمة,AL MADINAH...,"['Salary','30000']",دوام جزئي,0 / 40,FRAUD2025BENCONT1012,1 Years,NaN
2014,1,طبيب,29/05/1444,['فرصة عمل مميزة براتب مغري.'],"[' ترتيب المنتجات', ' خدمة الزبائن', ' م...",مؤسسة النجاح الفوري,4-178257,غير ربحي,متوسطة فئة أ,Healthcare,NaN,المنطقة الشرقية,AL MADINAH...,"['Salary','15000']",دوام كامل,0 / 8,FRAUD2025ECO1018,3 Years,NaN


Fake rows number 553


### Check of merge 5 fake - 5 real

In [4]:
import pandas as pd

df = pd.read_csv("mergedFakeWithRealData.csv", encoding="utf-8-sig")

# 
fake_head = df[df["fraudulent"] == 1].head(5).copy()
fake_head["Type"] = "Fake"

real_head = df[df["fraudulent"] == 0].head(5).copy()
real_head["Type"] = "Real"

sample_df = pd.concat([fake_head, real_head], ignore_index=True)

# display no save
sample_df


,fraudulent,job_title,job_date,job_desc,job_tasks,comp_name,comp_no,comp_type,comp_size,eco_activity,qualif,region,city,benefits,contract,positions,job_post_id,exper,gender,Type
0,1,خدمة عملاء,27/05/1444,['فرصة عمل مميزة براتب مغري.'],"[' وضع استراتيجيات طويلة الأمد', ' تطوير خ...",جهة حكومية سرية,5-292433,حكومي سري,كبيرة,NaN,NaN,الرياض,JEDDAH...,"['Salary','30000']",عقد مؤقت,0 / 35,FRAUD2025CONTASK1045,1 Years,NaN,Fake
1,1,كاتب محتوى,30/05/1444,['وظيفة عاجلة - تواصل عبر البريد الشخصي.'],"[' ترتيب المنتجات', ' خدمة الزبائن', ' م...",جهة حكومية سرية,1-725396,حكومي سري,صغيرة فئة أ,Education,NaN,المنطقة الشرقية,AD DAMMAM...,"['Salary','20000']",دوام كامل,0 / 3,FRAUD2025ECO1015,1 Years,NaN,Fake
2,1,مسوق إلكتروني,27/05/1444,['راتب عالي جدا بدون أي متطلبات إضافية.'],"[' فحص المرضى', ' كتابة تقارير طبية', ' ...",مؤسسة النجاح الفوري,9-878394,حكومي سري,كبيرة,Information Technology,NaN,المنطقة الشرقية,AD DAMMAM...,"['Salary','30000']",دوام كامل,0 / 12,FRAUD2025ECO1006,0 Years,NaN,Fake
3,1,سكرتير,01/01/1438,['فرصة محدودة - التحق فوراً قبل إغلاق التقديم.'],"[' الرد على رسائل واتساب', ' حفظ ملفات بسي...",جهة حكومية سرية,3-698181,خاص,صغيرة فئة أ,NaN,NaN,مكة المكرمة,AR RIYADH...,"['Salary','15000']",دوام كامل,0 / 18,FRAUD2025DATEDESC1040,0 Years,NaN,Fake
4,1,محاسب,27/05/1444,['فرصة عمل مميزة براتب مغري.'],"[' الرد على رسائل واتساب', ' حفظ ملفات بسي...",جهة حكومية سرية,1-614106,خاص,متوسطة فئة أ,NaN,NaN,الرياض,AL MADINAH...,"['Salary','90000']",عقد مؤقت,0 / 13,FRAUD2025SIZEBEN1034,1 Years,NaN,Fake
5,0,مشرف تنظيف وتدبير,07/05/1444,['الإشراف على أنشطة التدبير في المرافق وتنسيقه...,[' إدارة الجدول الزمني للخدمات. التخطيط للجد...,مجموعةالذيابي للمقاولات,1-4320,خاص,متوسطة فئة ج,الانشاءات العامة للمباني غير السكنية (مثل المد...,NaN,الرياض,AR RIYADH...,"['Salary', '4000.0']",دوام كامل,0 / 2,20202026315617,0 Years,both,Real
6,0,بناء,02/06/1444,['المشاركة الفعالة في عمليات البناء وفقا للمخط...,[' المشاركة في تحضير الموقع لتشييد المباني، ...,شركة عبدالله محمد اليوسف للمقاولات,15-1261960,خاص,متوسطة فئة ج,أنشطة خدمات صيانة المباني,NaN,المنطقة الشرقية,AL KHUBAR...,"['Salary', '4000.0']",دوام كامل,0 / 1,20202026368158,0 Years,M,Real
7,0,بائع مأكولات ومشروبات,20/05/1444,['بيع المأكولات و المشروبات للزبائن، وتوفير ال...,"[' بيع المأكولات و المشروبات للزبائن.', ' ت...",شركة الفصل الخامس للتجارة,2-1963632,خاص,متوسطة فئة أ,بيع الأغذية والمشروبات بالتجزئة في الأكشاك وال...,NaN,الرياض,AL KHARJ...,"['Salary', '4500.0']",دوام كامل,0 / 4,20202026341782,0 Years,M,Real
8,0,مهندس كهربائي,03/05/1444,['إعداد التصاميم والمخططات الكهربائية لمشاريع ...,[' إعداد التصاميم والمخططات الكهربائية لمشار...,شركة مجموعة الفارس التجارية (شركة شخص واحد),11-1546164,خاص,كبيرة,أنشطة خدمات صيانة المباني,"['Professional certificate \\ License data', '...",عسير,ABHA...,"['Salary', '7500.0']",دوام كامل,0 / 1,20202026311461,2 Years,M,Real
9,0,معلم ابتدائي علوم ورياضيات,04/05/1444,['تخطيط وتحضير الدروس المتعلقة بالعلوم والرياض...,['تحضير الدروس الخاصة بالعلوم والرياضيات وفقاً...,شركة مدارس موسوعة العلوم الاهلية,1-383334,خاص,متوسطة فئة أ,التعليم المتوسط للطالبات بمنهج وطني,"['Skill data', 'استخدام أدوات التكنولوجيا في ا...",الرياض,AR RIYADH...,"['Salary', '6000.0', 'Benefits', 'خصم أبناء 40...",دوام كامل,0 / 1,20202026311976,4 Years,F,Real


### ADD "type" colmn

In [5]:
import pandas as pd
import numpy as np

df = pd.read_csv("mergedFakeWithRealData.csv", encoding="utf-8-sig")


if "fraudulent" not in df.columns:
    raise ValueError("عمود 'fraudulent' غير موجود في الملف.")


df["Type"] = np.where(df["fraudulent"] == 1, "Fake",
              np.where(df["fraudulent"] == 0, "Real", "Unknown"))


df.to_csv("mergedFakeWithRealData.csv", index=False, encoding="utf-8-sig")


df.head()


,fraudulent,job_title,job_date,job_desc,job_tasks,comp_name,comp_no,comp_type,comp_size,eco_activity,qualif,region,city,benefits,contract,positions,job_post_id,exper,gender,Type
0,0,مشرف تنظيف وتدبير,07/05/1444,['الإشراف على أنشطة التدبير في المرافق وتنسيقه...,[' إدارة الجدول الزمني للخدمات. التخطيط للجد...,مجموعةالذيابي للمقاولات,1-4320,خاص,متوسطة فئة ج,الانشاءات العامة للمباني غير السكنية (مثل المد...,NaN,الرياض,AR RIYADH...,"['Salary', '4000.0']",دوام كامل,0 / 2,20202026315617,0 Years,both,Real
1,0,بناء,02/06/1444,['المشاركة الفعالة في عمليات البناء وفقا للمخط...,[' المشاركة في تحضير الموقع لتشييد المباني، ...,شركة عبدالله محمد اليوسف للمقاولات,15-1261960,خاص,متوسطة فئة ج,أنشطة خدمات صيانة المباني,NaN,المنطقة الشرقية,AL KHUBAR...,"['Salary', '4000.0']",دوام كامل,0 / 1,20202026368158,0 Years,M,Real
2,0,بائع مأكولات ومشروبات,20/05/1444,['بيع المأكولات و المشروبات للزبائن، وتوفير ال...,"[' بيع المأكولات و المشروبات للزبائن.', ' ت...",شركة الفصل الخامس للتجارة,2-1963632,خاص,متوسطة فئة أ,بيع الأغذية والمشروبات بالتجزئة في الأكشاك وال...,NaN,الرياض,AL KHARJ...,"['Salary', '4500.0']",دوام كامل,0 / 4,20202026341782,0 Years,M,Real
3,0,مهندس كهربائي,03/05/1444,['إعداد التصاميم والمخططات الكهربائية لمشاريع ...,[' إعداد التصاميم والمخططات الكهربائية لمشار...,شركة مجموعة الفارس التجارية (شركة شخص واحد),11-1546164,خاص,كبيرة,أنشطة خدمات صيانة المباني,"['Professional certificate \\ License data', '...",عسير,ABHA...,"['Salary', '7500.0']",دوام كامل,0 / 1,20202026311461,2 Years,M,Real
4,0,معلم ابتدائي علوم ورياضيات,04/05/1444,['تخطيط وتحضير الدروس المتعلقة بالعلوم والرياض...,['تحضير الدروس الخاصة بالعلوم والرياضيات وفقاً...,شركة مدارس موسوعة العلوم الاهلية,1-383334,خاص,متوسطة فئة أ,التعليم المتوسط للطالبات بمنهج وطني,"['Skill data', 'استخدام أدوات التكنولوجيا في ا...",الرياض,AR RIYADH...,"['Salary', '6000.0', 'Benefits', 'خصم أبناء 40...",دوام كامل,0 / 1,20202026311976,4 Years,F,Real


In [6]:
import pandas as pd

df = pd.read_csv("mergedFakeWithRealData.csv", encoding="utf-8-sig")

df = df.drop(columns=["type"], errors="ignore")

df.to_csv("mergedFakeWithRealData.csv", index=False, encoding="utf-8-sig")

print(df.head())


   fraudulent                   job_title    job_date  \
0           0           مشرف تنظيف وتدبير  07/05/1444   
1           0                        بناء  02/06/1444   
2           0       بائع مأكولات ومشروبات  20/05/1444   
3           0               مهندس كهربائي  03/05/1444   
4           0  معلم ابتدائي علوم ورياضيات  04/05/1444   

                                            job_desc  \
0  ['الإشراف على أنشطة التدبير في المرافق وتنسيقه...   
1  ['المشاركة الفعالة في عمليات البناء وفقا للمخط...   
2  ['بيع المأكولات و المشروبات للزبائن، وتوفير ال...   
3  ['إعداد التصاميم والمخططات الكهربائية لمشاريع ...   
4  ['تخطيط وتحضير الدروس المتعلقة بالعلوم والرياض...   

                                           job_tasks  \
0  ['   إدارة الجدول الزمني للخدمات. التخطيط للجد...   
1  ['   المشاركة في تحضير الموقع لتشييد المباني، ...   
2  ['   بيع المأكولات و المشروبات للزبائن.', '  ت...   
3  ['   إعداد التصاميم والمخططات الكهربائية لمشار...   
4  ['تحضير الدروس الخاصة بالعلوم والرياض

### drop "job_post_id", "positions", "comp_no"

In [7]:
import pandas as pd

df = pd.read_csv("mergedFakeWithRealData.csv", encoding="utf-8-sig")

df = df.drop(columns=["job_post_id", "positions", "comp_no"])

print(df.shape)
print(df.head())

# update to merged CSV
df.to_csv("mergedFakeWithRealData.csv", index=False, encoding="utf-8-sig")


(2023, 17)
   fraudulent                   job_title    job_date  \
0           0           مشرف تنظيف وتدبير  07/05/1444   
1           0                        بناء  02/06/1444   
2           0       بائع مأكولات ومشروبات  20/05/1444   
3           0               مهندس كهربائي  03/05/1444   
4           0  معلم ابتدائي علوم ورياضيات  04/05/1444   

                                            job_desc  \
0  ['الإشراف على أنشطة التدبير في المرافق وتنسيقه...   
1  ['المشاركة الفعالة في عمليات البناء وفقا للمخط...   
2  ['بيع المأكولات و المشروبات للزبائن، وتوفير ال...   
3  ['إعداد التصاميم والمخططات الكهربائية لمشاريع ...   
4  ['تخطيط وتحضير الدروس المتعلقة بالعلوم والرياض...   

                                           job_tasks  \
0  ['   إدارة الجدول الزمني للخدمات. التخطيط للجد...   
1  ['   المشاركة في تحضير الموقع لتشييد المباني، ...   
2  ['   بيع المأكولات و المشروبات للزبائن.', '  ت...   
3  ['   إعداد التصاميم والمخططات الكهربائية لمشار...   
4  ['تحضير الدروس الخاصة بالع